In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pickle



In [20]:
# Load the dataset
df = pd.read_csv('2025_train.csv', low_memory=False)


df['year_index'] = pd.Series(df['academicYear']).rank(method='dense').astype(int)

# Set academicYear as index if needed
df.set_index('year_index', inplace=True)

df_2019 = df.loc[1]  # Based on your date format
print("DataFrame with year index 2019:" ,df_2019.head())

DataFrame with year index 2019:             academicYear  universityName  faculty  departmentName  \
year_index                                                          
1                   2019               0      194             298   
1                   2019               0      201              36   
1                   2019               0      201              94   
1                   2019               0      201             107   
1                   2019               0      201             278   

                 idOSYM  scholarshipRate  quota  occupiedSlots  tuitionFee  \
year_index                                                                   
1           106510014.0                0   62.0           62.0         0.0   
1           106510077.0                0   62.0           62.0         0.0   
1           106510068.0                0   62.0           62.0         0.0   
1           106510023.0                0   62.0           62.0         0.0   
1           1065

### Encoding

In [12]:
binary_features = [
    'universityType_devlet', 'universityType_vakıf', 'programType_DİL', 'programType_EA', 
    'programType_SAY', 'programType_SÖZ', 'language_Almanca', 'language_Arapça', 'language_Bulgarca',  
    'language_Ermenice', 'language_Fransızca', 'language_Korece', 'language_Lehçe', 'language_Rusça', 
    'language_Türkçe', 'language_Çince', 'language_İngilizce', 'language_İspanyolca', 'language_İtalyanca', 
    'baseRanking','idOSYM_flag', 'academicYear_flag'
]
print(df[binary_features].describe().round(3))
print(len(binary_features))



       universityType_devlet  universityType_vakıf  programType_DİL  \
count              50554.000             50554.000        50554.000   
mean                   0.607                 0.393            0.049   
std                    0.488                 0.488            0.215   
min                    0.000                 0.000            0.000   
25%                    0.000                 0.000            0.000   
50%                    1.000                 0.000            0.000   
75%                    1.000                 1.000            0.000   
max                    1.000                 1.000            1.000   

       programType_EA  programType_SAY  programType_SÖZ  language_Almanca  \
count       50554.000        50554.000        50554.000         50554.000   
mean            0.324            0.453            0.174             0.004   
std             0.468            0.498            0.379             0.063   
min             0.000            0.000            0.

In [13]:
numeric_features = []
for feature in df.columns:
    if feature not in binary_features:
        numeric_features.append(feature)
        
print(len(numeric_features))

42


In [14]:
skew_values = df[numeric_features].skew()

skew_features = skew_values[abs(skew_values) > 1.5].index.tolist()

print("Skewed Features:", skew_features)
print(len(skew_features))

continuous_features = [col for col in numeric_features if col not in skew_features]

print("Continous Features:", continuous_features)
print(len(continuous_features))

Skewed Features: ['quota', 'occupiedSlots', 'tuitionFee', 'profCount', 'assoCount', 'docCount', 'topRanking', 'avgAdmissionRanking(TYT)', 'stdDeviationStudents', 'revenue', 'totalPreference', 'top1PreferenceRatio', 'avgAdmittedStudentPrefOrder', 'top1AdmittedRatio', 'top10AdmittedRatio', 'admittedTotalPref', 'admittedTotalDepartmentPref', 'currentStudentCount', 'baseScore', 'topScore', 'totalForeignStudents']
21
Continous Features: ['academicYear', 'universityName', 'faculty', 'departmentName', 'idOSYM', 'scholarshipRate', 'universityLocation', 'universityRegion', 'baseAdmissionRanking(TYT)', 'outOfCityStudentRate', 'avgOrderofPreference', 'top3AdmittedRatio', 'totalStudentNumber', 'Urap_Rank', 'Urap_Score', 'avg_monthly_income_group', 'Time_for_employment', 'employment_rate', 'base_salary_by_year', 'inflation_by_year', 'growth_by_year']
21


In [17]:
scaler_standard = StandardScaler()

# Step 1: Apply log transformation to skewed features
df[skew_features] = df[skew_features] + 0.1  # Add small constant to avoid log(0)
df[skew_features] = np.log1p(df[skew_features])

# Step 2: Combine all features into a single dataset
numeric_features = continuous_features + skew_features

# Step 3: Change to int
df[numeric_features] = df[numeric_features].astype(float)

# Step 3: Apply the same scaling to all features
preprocessor = ColumnTransformer(
    transformers=[
        ('all', scaler_standard, numeric_features),
        ('binary', 'passthrough', binary_features)
    ]
)

# Fit and transform the data
processed_data = preprocessor.fit_transform(df)

# Convert the result back to a DataFrame
processed_df = pd.DataFrame(processed_data, columns=numeric_features+binary_features)


# Print descriptive statistics
print(processed_df.describe().round(3))

       academicYear  universityName    faculty  departmentName     idOSYM  \
count     50554.000       50554.000  50554.000       50554.000  50554.000   
mean         -0.000          -0.000      0.000          -0.000      0.000   
std           1.000           1.000      1.000           1.000      1.000   
min          -1.635          -1.765     -2.294          -1.869     -0.921   
25%          -1.035          -0.914     -1.104          -0.881     -0.823   
50%           0.166           0.053     -0.159          -0.101     -0.728   
75%           0.766           0.904      0.615           0.887      1.209   
max           1.367           1.637      2.191           1.778      1.369   

       scholarshipRate  universityLocation  universityRegion  \
count        50554.000           50554.000         50554.000   
mean             0.000              -0.000            -0.000   
std              1.000               1.000             1.000   
min             -0.678              -1.707        

In [18]:
processed_df.to_csv('dataset_encoded_scaled.csv', index=False)

In [19]:
## Print data types
print(processed_df.dtypes)

academicYear           float64
universityName         float64
faculty                float64
departmentName         float64
idOSYM                 float64
                        ...   
language_İspanyolca    float64
language_İtalyanca     float64
baseRanking            float64
idOSYM_flag            float64
academicYear_flag      float64
Length: 64, dtype: object
